In [61]:
import credentials # Import api/access_token keys from credentials.py
import re
import tweepy
import settings
import mysql.connector
import pandas as pd
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
import preprocessor as p

# p.clean('Preprocessor is #awesome 👍 https://github.com/s/preprocessor')

class MyStreamListener(tweepy.StreamListener):

    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

    def lemma(comment):
        doc = MyStreamListener.nlp(comment)
        return " ".join([token.lemma_ for token in doc])    

    def deEmojify(self, text):
        regrex_pattern = re.compile(pattern = "["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            "]+", flags = re.UNICODE)
        return regrex_pattern.sub(r'',text)

    def preprocess(self, tweet):    
        text = re.sub(r'http\S+', '', tweet.lower(), flags=re.MULTILINE)
        res = re.sub(r'[^\w\s]', '', text)
        deemo = self.deEmojify(res)
        le = self.lemma(deemo)
        return le

    def on_status(self, status):
        if status.retweeted:
        #Avoid retweeted info, and only original tweets will be received
            return True
        # Extract info from tweets
        id_str = status.id_str
        created_at = status.created_at
        # text = self.preprocess(status.text)    # Pre-processing the text  
        text = p.clean(status.text)
        sentiment = TextBlob(text).sentiment
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        doc = nlp(text)
        enti = str()
        cond = ['PERSON', 'GPE', 'ORG']
        for ent in doc.ents: 
            if ent.label_ in cond:
                enti = enti + "," + ent.text                 
        enti = enti[1:]
        user_created_at = status.user.created_at
        user_location = self.deEmojify(status.user.location)
        user_description = self.deEmojify(status.user.description)
        user_followers_count =status.user.followers_count
        longitude = None
        latitude = None
        if status.coordinates:
            longitude = status.coordinates['coordinates'][0]
            latitude = status.coordinates['coordinates'][1]
        
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count
        
        print(status.text)
        print(text)
        print("Long: {}, Lati: {}".format(longitude, latitude))

        if mydb.is_connected():
            mycursor = mydb.cursor()
            sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity, named_ent, user_created_at, user_location, user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(settings.TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity, enti, user_created_at, user_location, \
                user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
            mycursor.execute(sql, val)
            mydb.commit()
            mycursor.close()


    def on_error(self, status_code):
        '''
        Since Twitter API has rate limits, 
        stop srcraping data as it exceed to the thresold.
        '''
        if status_code == 420:
            # return False to disconnect the stream
            return False


In [2]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
# nltk.download('stopwords')

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def preprocess(tweet):    
    text = re.sub(r'http\S+', '', tweet.lower(), flags=re.MULTILINE)
    print(text)
    res = re.sub(r'[^\w\s]', '', text)
    print(res)
    deemo = deEmojify(res)
    print(deemo)
    le = lemma(deemo)
    print(le)
    return le

def lemma(comment):
    doc = nlp(comment)
    return " ".join([token.lemma_ for token in doc])    

In [3]:
print(lemma("Hello can't you do this?"))

hello ca n't you do this ?


In [59]:
print(preprocess("Hello Parth #happy"))

hello parth #happy
hello parth happy
hello parth happy
hello parth happy
hello parth happy


In [5]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="password",
    database="TwitterDB"
)
if mydb.is_connected():
    '''
    Check if this table exits. If not, then create a new one.
    '''
    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(settings.TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(settings.TABLE_NAME, settings.TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()

InterfaceError: 2003: Can't connect to MySQL server on 'localhost:3306' (10061 No connection could be made because the target machine actively refused it)

In [13]:
from sqlite3 import OperationalError

def check_conn(conn):
    try:
        conn.cursor()
        return True
    except Exception as ex:
        return False

myconn =sqlite3.connect('twitter.db')

if check_conn(myconn) == True:        
    mycursor = myconn.cursor()
    try:
        output=myconn.execute("SELECT COUNT(*) FROM {}".format(settings.TABLE_NAME))
        result=output.fetchall()
    except OperationalError:
        myconn.execute("CREATE TABLE {} ({})".format(settings.TABLE_NAME, settings.TABLE_ATTRIBUTES))
        myconn.commit()

    myconn.close()

In [ ]:
auth = tweepy.OAuthHandler(credentials.consumer_key credentials.consumer_secret)
auth.set_access_token(credentials.access_token, credentials.access_token_secret)    
api = tweepy.API(auth,wait_on_rate_limit=True)


In [ ]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
myStream.filter(languages=["en"], track = settings.TRACK_WORDS)
# Close the MySQL connection as it finished
# However, this won't be reached as the stream listener won't stop automatically
# Press STOP button to finish the process.
myconn.close()

In [8]:
import spacy

nlp = spacy.load('en_core_web_sm') 

sentence = "Hello, my name is Parth and I live in Mumbai. I love eating cake and I was in Madrid last week. I use Microsoft's Laptop. Audi is a company that will grow a lot."

doc = nlp(sentence)

enti = str()
cond = ['PERSON', 'GPE', 'ORG']
for ent in doc.ents: 
	if ent.label_ in cond:
		enti = enti + "," + ent.text
		print(ent.text, ent.start_char, ent.end_char, ent.label_) 

enti = enti[1:]
print(enti)

Parth 18 23 PERSON
Mumbai 38 44 GPE
Madrid 78 84 GPE
Microsoft 102 111 ORG
Audi 122 126 ORG
Parth,Mumbai,Madrid,Microsoft,Audi


In [16]:
import re
text = re.findall("@([a-zA-Z0-9_]+)", "https://t.co/0zyatHjUMa  @ImranKhanPTI @PTIofficial @pmln_org  question for all of you regarding taxes and level of collection")
print(text)



['ImranKhanPTI', 'PTIofficial', 'pmln_org']


In [5]:
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
import os

java_path = "C:/Program Files/Java/jdk1.8.0_131/bin/java.exe"
os.environ['JAVAHOME'] = java_path

st = StanfordNERTagger('D:/Parth/Downloads/Stanford NER files/stanford-ner-2020-11-17/classifiers/english.all.3class.distsim.crf.ser.gz',
					   'D:/Parth/Downloads/Stanford NER files/stanford-ner-2020-11-17/stanford-ner.jar',
					   encoding='utf-8')

text = 'While in France, Christine Lagarde discussed short-term stimulus efforts in a recent interview with the Wall Street Journal.'

tokenized_text = word_tokenize(text)
classified_text = st.tag(tokenized_text)


print(classified_text[2][1])

LOCATION


In [6]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="myGeocoder")

location = geolocator.geocode("At your local Costco")

print((location.latitude, location.longitude))

AttributeError: 'NoneType' object has no attribute 'latitude'

In [ ]:
'''
Further steps:

1) Extract country name and country code from location and add exception
2) Calculate the total number of occurences of country 
3) Combine all the data into a DataFrame and plot
'''